Usaremos a API SerpApi, que é usada para realizar o web scraping em engines de buscas(neste caso o Google). Essa API lida com as dificuldades na coleta de dados dessas ferramentas de busca, como conteúdo dinâmico, proxies e CAPTCHAs.

In [1]:
import serpapi
import os
import time
import pandas as pd
import numpy as np

#criando um objeto client com a chave da api para realizer as pesquisas
api_key = ''
client = serpapi.Client(api_key=api_key)

In [2]:
#configurando a pesquisa que será realizada 
results = client.search({
    'engine':'google_maps', #define a api que será usada
    'type':'search', #define o tipo de busca
    'q':'the ship London' #define o termo que será buscado
})

In [3]:
#precisamos pegar o id do estabalecimento que analisaremos
place_info = results.get("local_results")
place_id = place_info[0].get('place_id')
print(place_id)

ChIJkxWeZ30PdkgRZiJi42dFC8o


In [4]:
NUM_REVIEWS = 100 #definindo o número de reviews

#definindo novos parâmetros para a pesquisa utilizando a nova api e o código do estabalecimento
params = {
    'engine':'google_maps_reviews',
    'type':'search',
    'place_id': place_id
}

reviews = []

while len(reviews) < NUM_REVIEWS:
    reviews_results = client.search(params)

    if "reviews" in reviews_results:
        r = reviews_results.get("reviews")
        reviews.extend(r) 
    
    if "serpapi_pagination" in reviews_results and "next" in reviews_results["serpapi_pagination"]:
        params["next_page_token"] = reviews_results["serpapi_pagination"]["next_page_token"] #caso exista uma nova página, adicionaremos o token aos parâmetros para ela ser acessada em seguida
    else:
        break #caso não exista uma nova página, o looping é encerrado

    time.sleep(2)

print(len(reviews))

108


In [5]:
data = []

#loopinh para extrair o rating e a review do cliente
for i in range(len(reviews)):
    rating = reviews[i].get("rating")
    text = reviews[i].get("snippet")

    current_review = {
        "rating": rating,
        "text": text
    }

    data.append(current_review)

#criação do DataFrame
df = pd.DataFrame(data = data)
df.head(5)

,rating,text
0,4.0,Went on a sunny day for a Sunday roast with fr...
1,5.0,"I recently visited **The Ship**, a charming sp..."
2,4.0,Stopped by for a pint and a quick lunch. It's ...
3,5.0,A solid pub on the bank of the Thames you can ...
4,5.0,Amazing summer place to have sunday roast of c...


In [6]:
df['text'] = df['text'].apply(lambda x: x.replace('\n', '')) #aplicação da função map utilizando uma função anônima para tratar os dados
df['id'] = df.index + 1
df = df.reindex(columns=['id', 'rating', 'text'])
reviews = df.rename(columns={'id':'Id', 'rating':'Score', 'text':'Text'})

reviews

,Id,Score,Text
0,1,4.0,Went on a sunny day for a Sunday roast with fr...
1,2,5.0,"I recently visited **The Ship**, a charming sp..."
2,3,4.0,Stopped by for a pint and a quick lunch. It's ...
3,4,5.0,A solid pub on the bank of the Thames you can ...
4,5,5.0,Amazing summer place to have sunday roast of c...
...,...,...,...
103,104,4.0,Nice bar for an Saturday evening. Interior loo...
104,105,4.0,Outside in the sun on one of the benches on a ...
105,106,1.0,What is happening? Standing at the till with a...
106,107,5.0,Oh yea this place is a vibe. Can't wait to com...


In [7]:
reviews.to_csv("the_ship_reviews.csv", index=False)